Test notebook to practice reading s3 files to SageMaker. The role returned by get_execution_role() needs to be added to the s3 Bucket policy within the Principal Key as "AWS":"arn:aws:iam::144273247162:role/SageMakerExecutionRole"

Currently the bucket is also allowing public access, so I'm not sure if this will work if the data is made private. However, I ran into an error when trying to access the public bucket without adding this role to the Policy.

## Imports

In [1]:
import os
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
import boto3
import sagemaker
from sagemaker import get_execution_role
import tempfile
import io

In [2]:
role = get_execution_role()
print(role)

region = boto3.Session().region_name
sess = sagemaker.Session()

arn:aws:iam::144273247162:role/SageMakerExecutionRole


In [3]:
bucket = 'cis5190-foodreviews'
file_path = 'Reviews_cleanText_noSW.csv'

s3_client = boto3.client('s3')
contents = s3_client.list_objects(Bucket=bucket)['Contents']

In [4]:
for f in contents:
    print(f['Key'])

Reviews_cleanText_noSW.csv
Reviews_cleanText_sw.csv


In [5]:
obj = s3_client.get_object(Bucket=bucket, Key=file_path)

df = pd.read_csv(io.BytesIO(obj['Body'].read()))
df.head()

,Text,Summary,Score,cleaned_text,cleaned_summary,Score_class
0,I have bought several of the Vitality canned d...,Good Quality Dog Food,5,buy several vitality can dog food products fin...,good quality dog food,1
1,Product arrived labeled as Jumbo Salted Peanut...,Not as Advertised,1,product arrive label jumbo salt peanutsthe pea...,advertise,-1
2,This is a confection that has been around a fe...,"""Delight"" says it all",4,confection around centuries light pillowy citr...,delight say,1
3,If you are looking for the secret ingredient i...,Cough Medicine,2,look secret ingredient robitussin believe find...,cough medicine,-1
4,Great taffy at a great price. There was a wid...,Great taffy,5,great taffy great price wide assortment yummy ...,great taffy,1


In [6]:
df.to_csv('Reviews_cleanText_noSW_sageMakerLocal.csv')